<a href="https://colab.research.google.com/github/karthikeyansura/store-sales-time-series-modeling/blob/main/store_sales_time_series_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>